# **Шаблоны улучшенных архитектур**

## Пакетная нормализация

В предыдущих примерах нормализация выполнялась перед передачей данных в модели. Однако нормализация должна проводиться после каждого преобразования,
выполняемого сетью: даже если данные на входе в сеть Dense или Conv2D имеют
среднее значение 0 и единичную дисперсию, нет оснований полагать, что то же
самое можно будет сказать в отношении данных на выходе.

Пакетная нормализация — это тип слоя (BatchNormalization в Keras), введенный в 2015 году Сергеем Йоффе (Sergey Ioffe) и Кристианом Сегеди (Christian
Szegedy)1
; он может адаптивно нормализовать данные, даже если среднее и дисперсия изменяются во время обучения. Его принцип действия основан на вычислении экспоненциального скользящего среднего и дисперсии данных, наблюдаемых
в процессе обучения. Основное назначение пакетной нормализации — помочь
распространению градиента подобно остаточным связям и дать возможность
создавать более глубокие сети. Некоторые глубокие сети могут обучаться, только
если они включают в себя несколько слоев BatchNormalization. Например, слои
BatchNormalization широко используются во многих продвинутых архитектурах сверточных нейронных сетей, входящих в состав Keras, таких как ResNet50,
Inception V3 и Xception.

Обычно слой BatchNormalization используется после сверточного или полносвязного слоя:

In [ ]:
conv_model.add(layers.Conv2D(32, 3, activation='relu'))
conv_model.add(layers.BatchNormalization())               # После слоя Conv

dense_model.add(layers.Dense(32, activation='relu'))
dense_model.add(layers.BatchNormalization())              # После слоя Dense


Слой `BatchNormalization` принимает аргумент `axis`, определяющий ось признаков для нормализации. По умолчанию этот аргумент принимает значение `–1`, что соответствует последней оси во входном тензоре. Это правильное значение,
когда используются слои `Dense, Conv1D`, рекуррентные слои и слои `Conv2D` со значением `"channels_last"` в аргументе `data_format`. Но в слоях `Conv2D` со значением
"`channels_first`" в аргументе `data_format` ось признаков — это ось с индексом 1;
поэтому в таких случаях слою `BatchNormalization` следует передавать число 1
аргументе axis.

## **ПАКЕТНАЯ РЕНОРМАЛИЗАЦИЯ**

Не так давно, в 2017 году, Сергеем Йоффе был предложен более совершенный метод, нежели обычная пакетная нормализация, — метод пакетной ренормализации1
.
Он предлагает очевидные преимущества по сравнению с пакетной нормализацией, без
дополнительных накладных расходов. На данный момент еще рано говорить, вытеснит
ли он пакетную нормализацию, но я думаю, что это вполне возможно. А совсем недавно
Гюнтер Кламбаер (Günter Klambauer) с коллегами представили самонормализующиеся
нейронные сети2
, которые позволяют сохранять данные в нормализованном состоянии
после прохождения через любой слой Dense за счет использования специальных функций активации (selu) и инициализации (lecun_normal). Эта схема выглядит довольно
интересной, но пока она ограничивается только полносвязными сетями и ее полезность
на данный момент не подтверждена широкими исследованиями.

## Раздельные свертки по глубине

Что бы вы подумали, если бы я сказал, что существует такой слой, который можно
использовать взамен Conv2D и с помощью которого сделать модель более легкой
(с меньшим количеством обучаемых весовых параметров) и быстрой (с меньшим
количеством операций с вещественными числами), а также повысить качество
решения задачи на несколько процентных пунктов? Всеми перечисленными качествами обладает слой раздельной свертки по глубине (SeparableConv2D). Этот
слой выполняет пространственную свертку каждого канала во входных данных
в отдельности перед смешиванием выходных каналов посредством поточечной
свертки (свертки 1 × 1), как показано на рис. 7.16. Это эквивалентно раздельному выделению пространственных и канальных признаков, что оправданно,
если предполагается сильная корреляция пространственных местоположений
на входе, но разные каналы практически не зависят друг от друга. Он требует
намного меньше параметров и выполняет меньше вычислений, благодаря чему
получаются более быстрые модели с меньшими размерами. И поскольку это более
репрезентативно эффективный способ выполнения свертки, он позволяет получать более качественные представления с меньшим объемом исходных данных
и, соответственно, более качественные модели.

Эти преимущества особенно важны для обучения небольших моделей с нуля на
ограниченном наборе данных. Например, вот как можно построить легковесную
нейронную сеть со сверткой по глубине для решения задачи классификации изображений (с применением классификатора softmax) при небольшом объеме обучающих данных:

In [ ]:
from keras.models import Sequential, Model
from keras import layers

height = 64
width = 64
channels = 3
num_classes = 10

model = Sequential()
model.add(layers.SeparableConv2D(32, 3,
                                activation='relu',
                                input_shape=(height, width, channels,)))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy')

В отношении крупных моделей раздельные свертки по глубине составляют основу
архитектуры Xception высококачественных сверточных нейронных сетей, входящей в состав Keras. Узнать больше о теоретических основах раздельной свертки
по глубине и архитектуре Xception можно в моей статье «Xception: Deep Learning
with Depthwise Separable Convolutions»1
.

# **Оптимизация гиперпараметров**

Проблемы сложны, а область еще молода, поэтому в настоящее время в нашем
распоряжении имеется весьма ограниченный набор инструментов для оптимизации моделей. Часто случайный поиск (многократный выбор случайных значений
гиперпараметров) оказывается лучшим решением, несмотря на то что он самый
простой. Однако я обнаружил один инструмент, который достоверно лучше случайного поиска, — Hyperopt (https://github.com/hyperopt/hyperopt), библиотеку
на Python для оптимизации гиперпараметров: внутренне она использует деревья
оценок Парзена для предсказания наборов гиперпараметров, которые с высокой
вероятностью дадут положительный результат. Еще одна библиотека, с названием
Hyperas (https://github.com/maxpumperla/hyperas), интегрирует Hyperopt для использования с моделями Keras. Обязательно обратите на нее внимание.

# **Ансамблирование моделей**

Возьмем в качестве примера задачу классификации. Самый простой способ объединить прогнозы из множества классификаторов (ансамблировать классификаторы) — получить среднее их прогнозов на этапе вывода:

In [ ]:
preds_a = model_a.predict(x_val)        #Применение четырех разных моделей для вычисления начальных прогнозов
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)

final_preds = 0.25 * (preds_a + preds_b + preds_c + preds_d) # Этот новый массив прогнозов должен получиться
                                                              # более точным, чем любой из начальных

Этот прием даст положительные результаты, только если исходные классификаторы примерно одинаково хороши. Если один будет значительно хуже других,
окончательный прогноз может получиться хуже прогноза лучшего классификатора
в группе.

Более эффективный способ ансамблирования классификаторов — вычисление
взвешенного среднего с определением весов по проверочным данным, когда лучший классификатор получает больший вес, а худший — меньший. Для поиска
оптимальных весов в ансамбле можно использовать алгоритм случайного поиска
или простой оптимизации, такой как **Nelder-Mead:**

In [ ]:
preds_a = model_a.predict(x_val)
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)

final_preds = 0.5 * preds_a + 0.25 * preds_b + 0.1 * preds_c + 0.15 * preds_d         #Эти веса (0.5, 0.25, 0.1, 0.15),
                                                                                        # как предполагается, получены эмпирическим путем


Существует много возможных вариантов: вы можете вычислить среднее экспоненциальное прогнозов, for instance. В общем случае простое взвешенное среднее
с весами, оптимизированными на проверочных данных, может служить очень неплохим базовым решением.

Ключом к достижению успеха в результате ансамблирования является разнообразие набора классификаторов. Разнообразие — это сила. Если бы все незрячие
мудрецы ощупали только хобот слона, они согласились бы, что слоны похожи на
змей, и навсегда бы остались в неведении о действительной форме слона. Разнообразие — вот что обеспечивает успех ансамблирования. Выражаясь языком
машинного обучения: если все ваши модели будут одинаково предвзятыми, ваш
ансамбль сохранит эту предвзятость. Если ваши модели будут предвзяты поразному, предвзятости будут нивелировать друг друга, и ансамбль получится
более точным и надежным.

По этой причине объединяться в ансамбли должны максимально хорошие и разные
модели. Это обычно означает использование разных архитектур или даже разных подходов к машинному обучению. Единственное, пожалуй, чего не следует
делать — ансамблировать ту же сеть, обученную несколько раз, даже при разных
начальных случайных значениях. Если ваши модели различаются только начальными значениями и тем, в каком порядке они обрабатывали обучающие данные,
ваш ансамбль будет иметь низкое разнообразие и обеспечит лишь незначительное
улучшение по сравнению с единственной моделью.

В своей практике я обнаружил один прием, дающий хорошие результаты, — однако он не является универсальным и подходит не для всякой предметной области — использование ансамбля древовидных методов (таких, как случайные
леса или деревья градиентного роста) и глубоких нейронных сетей. В 2014 году Андрей Колев (Andrei Kolev) и я вместе заняли четвертое место среди решений
задачи обнаружения бозона Хиггса на сайте Kaggle (www.kaggle.com/c/higgsboson), использовав ансамбль разных древовидных моделей и глубоких нейронных сетей. Примечательно, что одна из моделей в ансамбле реализовала другой
метод (это был регуляризованный жадный лес — regularized greedy forest) и имела
существенно худшую оценку, нежели остальные. Неудивительно, что она получила маленький вес в ансамбле. Тем не менее, к нашему удивлению, оказалось,
что она значительно улучшала ансамбль в целом, потому что сильно отличалась
от всех других моделей: она сохраняла информацию, к которой другие модели
не имели доступа. Именно в этом заключается суть ансамблирования. Главное
не то, насколько хороша ваша лучшая модель, а то, насколько разнообразны модели-кандидаты.

В последнее время большим успехом на практике пользовалась обширная категория моделей, сочетающих глубокое и поверхностное обучение. Такие модели
состоят из совместно обучаемых глубокой нейронной сети и большой линейной
модели. Совместное обучение семейства разных моделей — еще один способ ансамблирования.